进阶实战-05
====
**模板中动态调用模型数据**

# 模型重命名

在前面的练习中创建了一个模型---'Article'类。

由于html 5语言中有一个`<article>`标签， 使用`Article`类名，容易造成混乱和困扰，所以对这个模型重命名。

在django的管理界面中，模型数据表中的数据是按照数据录入顺序排列的，我希望它按照指定的字段---文章的撰写日期排序，并且是降序，即最近撰写的教程排在前面。

django的模型model实际就是定义django数据库中的一张表，修改模型名，也就是修改表名。

希望为文件信息表添加一个“副标题”字段。

编辑`tnt/models.py`文件，修改后的代码如下：

```
from django.db import models

# Create your models here.

class Note(models.Model):  #这里将类名修改为Note
    art_title = models.CharField(max_length=100)
    subhead = models.CharField(max_length=200)
    art_tag = models.CharField(max_length=50)
    second_tag = models.CharField(max_length=50) #添加的字段
    author = models.CharField(max_length=30)  #添加的字段
    art_abstract = models.TextField()
    first_date = models.DateField()
   
    def __str__(self):
        return self.art_title

    class Meta:  #添加一个子类 
        ordering = ['-first_date'] #依据first_date字段排序，逆排序。
```

关于django Model的 Meta选项的详细用法，请参见：[Model的Meta选项详解](../b-ModelLayer/Model的Meta选项.ipynb)

修改`admin.py`中的注册模型：

```
from django.contrib import admin

from .models import Note

# Register your models here.
admin.site.register(Note)
```

**重新生成迁移并应用迁移。**

启动服务，登录管理界面，原来的`Article`表，已更名为 `Note`表。

# 模板动态调用模型中的数据

在`Note`表中录入了一些记录，是已完成的文章的信息，这些信息已保存在数据库表中。

在模板中调用数据库中的数据，动态地呈现给访问者。

## 显示所有文章的列表

**修改模板文件**

打开`django-tnt/djangoindex.html`模板，在`<body>`标签中添加代码：
```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
</head>
<body>

{% for wen in allwen %}

        <h5>{{ forloop.counter }} -- {{ wen.art_title }} . {{ wen.first_date}}</h5> 

{% endfor %}

</body>
</html>
```

**修改视图**

编辑`tnt/views.py`文件，修改`djangoindex`视图函数如下：

```
from django.http import HttpRequest, HttpResponse
from django.shortcuts import render
from tnt.models import Note #注意要先导入Note类

# Create your views here.
def index(request):
    return render(request,'tnt/index.html')

def pandasindex(request):
    return render(request,'tnt/pandasindex.html')

def djangoindex(request): #修改这个视图函数
    allwen = Note.objects.all()
    return render(request,'django-tnt/djangoindex.html',{'allwen':allwen})
```

重新启动django服务，访问：http://127.0.0.1:8000/tnt/django/ 

就会列出`Note`表中所有文章，并按照`first_date`字段降序排列。

<img width="500px" src = "../../../pf-img-store/django/template-database01.png" >

# 显示分类的文章列表

修改模板，使用DTL的`{% if %}`标签，进行筛选，仅显示`art_tag`为"django"类别的文章：

```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
</head>
<body>

{% for wen in allwen %}
    {% ifequal wen.art_tag 'django' %}
        <h5>{{ forloop.counter }} -- {{ wen.art_title }} . {{ wen.first_date}}</h5> 
    {% endifequal %}
{% endfor %}

</body>
</html>
```

保存后，重新访问：http://127.0.0.1:8000/tnt/django/

<img width="500px" src = "../../../pf-img-store/django/template-database02.png" >

页面仅显示分类标签为"django"的文章的列表。

在模板中，使用一些特殊的标记调用数据库中的数据，并选择性地显示需要的数据。

先照着做一篇，理解django模型、数据库、模板协调工作机制，这是后面学习的基础。

至于django的模板语言DTL的语法，后面将详细讨论。